# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Seans Solutions

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kuta,-8.7233,115.1723,26.80,82,39,3.39,ID,1728860082
1,1,fale old settlement,-9.3852,-171.2468,27.97,77,67,5.71,TK,1728860083
2,2,waitangi,-43.9535,-176.5597,11.08,81,96,5.55,NZ,1728860084
3,3,adamstown,-25.0660,-130.1015,20.97,81,100,7.24,PN,1728860085
4,4,iskateley,67.6803,53.1512,1.14,95,70,6.43,RU,1728860086


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# %%capture --no-display
# Configure the map plot
map_plot= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles= "EsriImagery",
    size= "Humidity",
    frame_width= 700,
    frame_height= 500,
    scale= 0.5,
    color= "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# defining cities criteria
temp_min= city_data_df["Max Temp"] > 20
temp_max= city_data_df["Max Temp"] < 30
wind_speed= city_data_df["Wind Speed"] > 2

# Print
print(sum(temp_min))
print(sum(temp_max))
print(sum(wind_speed))

301
543
396


In [29]:
# Narrow down cities that fit criteria and drop any results with null values
# Max Temp
# Max Temp that is greater than 20 C and less than 30 C
# Wind speed greater than 7 mters per seconds (m/s) 
city_temp_range_df= city_data_df[temp_min & temp_max & wind_speed]
city_temp_range_df

# Drop any rows with null values
clean_city_temp_range_df= city_temp_range_df.dropna()
clean_city_temp_range_df
# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kuta,-8.7233,115.1723,26.80,82,39,3.39,ID,1728860082
1,1,fale old settlement,-9.3852,-171.2468,27.97,77,67,5.71,TK,1728860083
3,3,adamstown,-25.0660,-130.1015,20.97,81,100,7.24,PN,1728860085
18,18,enewetak,11.3474,162.3373,28.82,75,82,6.63,MH,1728860103
20,20,lihue,21.9789,-159.3672,28.77,65,20,5.66,US,1728860105
...,...,...,...,...,...,...,...,...,...,...
557,557,kirakira,-10.4544,161.9205,28.83,75,59,4.40,SB,1728860769
562,562,sarangani,5.4033,125.4636,29.23,74,64,6.09,PH,1728860775
565,565,long beach,33.7670,-118.1892,23.41,74,0,7.20,US,1728860485
570,570,say,13.1007,2.3689,29.23,63,100,4.05,NE,1728860785


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= clean_city_temp_range_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= "" 
hotel_df

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,kuta,-8.7233,115.1723,26.80,82,39,3.39,ID,1728860082,
1,1,fale old settlement,-9.3852,-171.2468,27.97,77,67,5.71,TK,1728860083,
3,3,adamstown,-25.0660,-130.1015,20.97,81,100,7.24,PN,1728860085,
18,18,enewetak,11.3474,162.3373,28.82,75,82,6.63,MH,1728860103,
20,20,lihue,21.9789,-159.3672,28.77,65,20,5.66,US,1728860105,
...,...,...,...,...,...,...,...,...,...,...,...
557,557,kirakira,-10.4544,161.9205,28.83,75,59,4.40,SB,1728860769,
562,562,sarangani,5.4033,125.4636,29.23,74,64,6.09,PH,1728860775,
565,565,long beach,33.7670,-118.1892,23.41,74,0,7.20,US,1728860485,
570,570,say,13.1007,2.3689,29.23,63,100,4.05,NE,1728860785,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 5000
limit = 1
categories= "hotel.accommodation"
params = {
    "categories": categories,
    "limit": limit,
    "APIKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng= hotel_df.loc[index, "Lng"]
    lat= hotel_df.loc[index, "Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(name_address.url)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=hotel.accommodation&limit=1&APIKey=eaecb15358034cab9bc9a61eb7cda281&filter=circle%3A115.1723%2C-8.7233%2C5000&bias=proximity%3A115.1723%2C-8.7233
kuta - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=hotel.accommodation&limit=1&APIKey=eaecb15358034cab9bc9a61eb7cda281&filter=circle%3A-171.2468%2C-9.3852%2C5000&bias=proximity%3A-171.2468%2C-9.3852
fale old settlement - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=hotel.accommodation&limit=1&APIKey=eaecb15358034cab9bc9a61eb7cda281&filter=circle%3A-130.1015%2C-25.066%2C5000&bias=proximity%3A-130.1015%2C-25.066
adamstown - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=hotel.accommodation&limit=1&APIKey=eaecb15358034cab9bc9a61eb7cda281&filter=circle%3A162.3373%2C11.3474%2C5000&bias=proximity%3A162.3373%2C11.3474
enewetak - nearest hotel: No hotel found
https://api.geoapify.com/v2/places

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,kuta,-8.7233,115.1723,26.80,82,39,3.39,ID,1728860082,No hotel found
1,1,fale old settlement,-9.3852,-171.2468,27.97,77,67,5.71,TK,1728860083,No hotel found
3,3,adamstown,-25.0660,-130.1015,20.97,81,100,7.24,PN,1728860085,No hotel found
18,18,enewetak,11.3474,162.3373,28.82,75,82,6.63,MH,1728860103,No hotel found
20,20,lihue,21.9789,-159.3672,28.77,65,20,5.66,US,1728860105,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
557,557,kirakira,-10.4544,161.9205,28.83,75,59,4.40,SB,1728860769,No hotel found
562,562,sarangani,5.4033,125.4636,29.23,74,64,6.09,PH,1728860775,No hotel found
565,565,long beach,33.7670,-118.1892,23.41,74,0,7.20,US,1728860485,No hotel found
570,570,say,13.1007,2.3689,29.23,63,100,4.05,NE,1728860785,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
# %%capture --no-display

# Configure the map plot
hotel_map_plot= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles= "EsriImagery",
    size= "Humidity",
    frame_width= 700,
    frame_height= 500,
    scale= 0.5,
    color= "City")

# Display the map
hotel_map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)